In [1]:
from STL_to_path import *
from GPT import *
from NL_to_STL import *
from visualization import *

         Install drake (https://drake.mit.edu/installation.html)
         to use the Drake-based solvers.


In [2]:

T_max = 10          # time horizon in seconds 
dt = 0.5            # time step in seconds
N = int(T_max/dt)   # number of time steps
max_acc = 50        # maximum acceleration in m/s^2
max_speed = 1       # maximum speed in m/s

objects = {"goal1": (4, 6, 4, 6, 4, 6),
           "goal2": (-5, -4, -5, -4, -5, -4),
           "goal3": (-6, -5, 4, 5, 4, 5),
           "obstacle1": (-1.5, -0.5, -1.5, -0.5, -1.5, -0.5),
           "obstacle2": (1.0, 2.0, 1.0, 2.0, 1.5, 2.5),
           "obstacle3": (1.0, 2.0, 0.0, 1.0, -1.0, 1.0),}

x0 = np.array([0.,0.,0.,0.,0.,0.]) # initial state: x, y, z, vx, vy, vz


In [3]:

user_input = [{"role": "user", "content": "Go to goal 1 and then to goal 2."},]


translator = NL_to_STL(objects, T_max, dt, print_instructions=True)
specs = translator.extract_specs(user_input)
print("specs: ", specs)


Instructions:  You are an assistant guiding a drone to perform an instruction specified by a user.
Your task is to take the user's instruction and translate it into a specification in Signal Temporal Logic (STL).
You are provided with a dictionary of objects in the scene, the available Logic functions, and Logic operators that you can use in your generated STL specification. All of these are provided below.

Objects:
{'goal1': (4, 6, 4, 6, 4, 6), 'goal2': (-5, -4, -5, -4, -5, -4), 'goal3': (-6, -5, 4, 5, 4, 5), 'obstacle1': (-1.5, -0.5, -1.5, -0.5, -1.5, -0.5), 'obstacle2': (1.0, 2.0, 1.0, 2.0, 1.5, 2.5), 'obstacle3': (1.0, 2.0, 0.0, 1.0, -1.0, 1.0)}

Logic functions: 
"STL_formulas.inside_cuboid(object)", "STL_formulas.outside_cuboid(object)"

Logic operators:
"&", "|"

STL specifications are represented as Python objects. some temporal functions operate on these obects. The syntax is like "specification.function()"
The available temporal functions are shown below.

Temporal functions

In [ ]:
def generate_trajectories(self, dt, max_acc, max_speed, verbose = False):
    self.dt = dt
    self.max_acc = max_acc
    self.max_speed = max_speed
    self.verbose = verbose
    N = int(self.T/self.dt)
    N_specs = len(self.specs)

    all_x = np.zeros((6, N_specs*(N+1)))
    all_u = np.zeros((3, N_specs*(N+1)))

    x0 = self.x0
    for i in range(N_specs):
        x, u = self.generate_trajectory(eval(self.specs[i]), x0)
        all_x[:,i*(N+1):(i+1)*(N+1)] = x
        all_u[:,i*(N+1):(i+1)*(N+1)] = u
        x0 = x[:,-1]
        print("x0: ", x0)

    return all_x, all_u

In [4]:

solver = STLSolver(specs, objects, x0, T_max)

x,u = solver.generate_trajectories(dt, max_acc, max_speed, verbose=True)

print("x: ", x)

print("u: ", u)


Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-29
Setting up optimization problem...
Setup complete in 0.4175705909729004 seconds.
<class 'gurobipy.MQuadExpr'>
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 900 rows, 523 columns and 1553 nonzeros
Model fingerprint: 0x06e47465
Model has 63 quadratic objective terms
Variable types: 451 continuous, 72 integer (72 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+03]
  Objective range  [1e+00, 1e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+100]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 590 rows and 220 columns
Presolve time: 0.00s

Explored 0 

TypeError: 'NoneType' object is not subscriptable

In [ ]:

animate = False
visualizer = Visualization(x, objects, animate=animate)

if animate:
    gif_name = input("Enter name of GIF file: ")
    visualizer.animate_trajectory(gif_name + ".gif")

visualizer.visualize()
visualizer.plot_distance_to_objects()

plt.show()